In [10]:
import os
import glob
from functools import reduce

In [11]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

# display plots in the notebook
%matplotlib inline

# increase default figure and font sizes for easier viewing
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

## Analysis
---

Description: This is a test for broker evaluation with varying number of subscribers and publishers. The VM hosting the broker is pinned to a sinlge core (core 9), and the broker process -- along with all its threads -- is pinned to a single core (core 3). The interval is set to 1ms (e.g., no. of pubs * 1000rps). 

In [12]:
results_dir = "../../../results/broker/vms/"
date_time_list = ["20170731121808.890320"]

cols = ['%cpu', '%mem'] 

In [13]:
broker_data_dict = {}

for date_time in date_time_list:
    for i in glob.glob(results_dir + date_time + "/broker_stats_with__*"):
        key = i.split('__')[1].split('_')[1] + "x" + i.split('__')[2].split('_')[1]        
        broker_data_dict[key] = pd.read_csv(i, sep=' ', header=None, na_filter=False, names = cols)

# sorted(broker_data_dict.keys())

In [14]:
kvm_data_dict = {}

for date_time in date_time_list:
    for i in glob.glob(results_dir + date_time + "/kvm_stats_with__*"):
        key = i.split('__')[1].split('_')[1] + "x" + i.split('__')[2].split('_')[1]        
        kvm_data_dict[key] = pd.read_csv(i, sep=' ', header=None, na_filter=False, names = cols)

# sorted(kvm_data_dict.keys())

In [15]:
vcpu_usage = pd.DataFrame()

cols = [2, 4, 8, 32, 64]

vcpu_usage['pubs'] = cols

for col in cols:
    row = []
    for i in vcpu_usage['pubs']:
        if str(i) + "x" + str(col) in broker_data_dict:
            row += [broker_data_dict[str(i) + "x" + str(col)]['%cpu'].mean()]
        else:
            row += [np.nan]
    vcpu_usage['%vcpu' + "-subs:" + str(col)] = row

vcpu_usage = vcpu_usage.set_index('pubs')
    
vcpu_usage

,%vcpu-subs:2,%vcpu-subs:4,%vcpu-subs:8,%vcpu-subs:32,%vcpu-subs:64
pubs,,,,,
2,11.426667,12.800000,15.320000,29.361538,49.304545
4,20.008333,22.218182,26.527273,53.790909,84.383333
8,35.109091,38.854545,46.872727,88.225000,NaN
32,98.316667,100.018182,NaN,NaN,NaN
64,99.872727,NaN,NaN,NaN,NaN


In [16]:
pcpu_usage = pd.DataFrame()

cols = [2, 4, 8, 32, 64]

pcpu_usage['pubs'] = cols

for col in cols:
    row = []
    for i in pcpu_usage['pubs']:
        if str(i) + "x" + str(col) in kvm_data_dict:
            row += [kvm_data_dict[str(i) + "x" + str(col)]['%cpu'].mean()]
        else:
            row += [np.nan]
    pcpu_usage['%pcpu' + "-subs:" + str(col)] = row

pcpu_usage = pcpu_usage.set_index('pubs')
    
pcpu_usage

,%pcpu-subs:2,%pcpu-subs:4,%pcpu-subs:8,%pcpu-subs:32,%pcpu-subs:64
pubs,,,,,
2,18.764286,19.968750,23.000000,36.258333,55.831818
4,28.852174,30.936364,35.663636,61.600000,87.172727
8,45.390909,48.890909,56.409091,91.654545,NaN
32,99.918182,99.890909,NaN,NaN,NaN
64,100.000000,NaN,NaN,NaN,NaN


In [17]:
vmem_usage = pd.DataFrame()

cols = [2, 4, 8, 32, 64]

vmem_usage['pubs'] = cols

for col in cols:
    row = []
    for i in vmem_usage['pubs']:
        if str(i) + "x" + str(col) in broker_data_dict:
            row += [broker_data_dict[str(i) + "x" + str(col)]['%mem'].mean()]
        else:
            row += [np.nan]
    vmem_usage['%vmem' + "-subs:" + str(col)] = row

vmem_usage = vmem_usage.set_index('pubs')
    
vmem_usage

,%vmem-subs:2,%vmem-subs:4,%vmem-subs:8,%vmem-subs:32,%vmem-subs:64
pubs,,,,,
2,0.1,0.1,0.1,0.1,0.2
4,0.2,0.1,0.1,0.1,0.2
8,0.1,0.1,0.1,0.1,NaN
32,0.1,0.1,NaN,NaN,NaN
64,0.1,NaN,NaN,NaN,NaN


In [18]:
pmem_usage = pd.DataFrame()

cols = [2, 4, 8, 32, 64]

pmem_usage['pubs'] = cols

for col in cols:
    row = []
    for i in pmem_usage['pubs']:
        if str(i) + "x" + str(col) in kvm_data_dict:
            row += [kvm_data_dict[str(i) + "x" + str(col)]['%mem'].mean()]
        else:
            row += [np.nan]
    pmem_usage['%pmem' + "-subs:" + str(col)] = row

pmem_usage = pmem_usage.set_index('pubs')
    
pmem_usage

,%pmem-subs:2,%pmem-subs:4,%pmem-subs:8,%pmem-subs:32,%pmem-subs:64
pubs,,,,,
2,5.5,5.5,5.5,5.5,5.5
4,5.5,5.5,5.5,5.5,5.5
8,5.7,5.7,5.7,5.7,NaN
32,5.7,5.7,NaN,NaN,NaN
64,5.7,NaN,NaN,NaN,NaN
